In [7]:
import torch
from transformers import CLIPTextProcessor, CLIPModel, CLIPImageProcessor, CLIPVisionModel

ImportError: cannot import name 'CLIPTextProcessor' from 'transformers' (/media/alain/Data/Tesis/torch/lib/python3.10/site-packages/transformers/__init__.py)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "openai/clip-vit-base-patch16"  # You can choose a different CLIP model if needed

model = CLIPVisionModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

In [6]:
model(torch.randn(2,3,224,224).cuda())[1].shape

torch.Size([2, 768])

In [8]:
words = ["red", "green", "0", "1"]
texts = [f"A photo of a {word}" for word in words]

# Create a placeholder for pixel values (even though we're not using actual images)
pixel_values = torch.zeros((len(words), 3, 224, 224)).to(device)

inputs = processor(texts, images=pixel_values, return_tensors="pt", padding=True, truncation=True).to(device)
outputs = model(**inputs)


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


In [9]:
vectors = outputs.last_hidden_state[:, 0, :]

AttributeError: 'CLIPOutput' object has no attribute 'last_hidden_state'

In [13]:
outputs.text_embeds.shape

torch.Size([4, 512])

In [72]:
import torch.nn as nn
from torch import sigmoid

class DictionaryClassifier(nn.Module):
    def __init__(self, dictionary):
        super(DictionaryClassifier, self).__init__()
        self.dictionary = dictionary # a tensor of shape (n x embed_dim)
        self.vocab_size, self.embed_dim = self.dictionary.shape
        self.cls = nn.Parameter(torch.ones(1,self.vocab_size, requires_grad=True))

    def forward(self, x, t=0.5): # x is an image
        similarities = x @ self.dictionary.T # Should be between 0 and 1
        hypo = sigmoid(self.cls)
        hypo = sigmoid(hypo)
        output = (similarities * hypo)[:,(hypo > t)[0]].prod(dim=1)
        return output

    def get_hypo_loss(self): # reduce 
        return sigmoid(self.cls).sum()
        
data = torch.randn(10,512).cuda()
model = DictionaryClassifier(outputs.text_embeds).cuda()
model(data).shape

torch.Size([10])

In [76]:
from torch.optim import Adam
from torch.nn.functional import binary_cross_entropy_with_logits as bce
opt = Adam(model.parameters(), lr=0.01)
l = 1.0 # Lambda for Hypo loss
x = torch.randn(10000,512).cuda()
y = (torch.randn(10000)>0).float().cuda()
n_iters = 1000
for i in range(n_iters):
    logits = model(x)
    opt.zero_grad()
    loss = bce(logits, y)
    loss_b = l*model.get_hypo_loss()
    loss_b.backward()
    opt.step()

    preds = (logits > 0.5).float()
    acc = (preds == y).float().mean()
    print(f"Loss: {loss:.3f}-Acc: {100*acc:.2f}%")

Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc: 49.48%
Loss: 0.714-Acc:

In [3]:
a = torch.load("../datasets/MNISTCIFAR/MNIST_CIFAR_binary_0.0.pth")

In [4]:
x = a['train']['data'].cuda()
y = a['train']['targets'].cuda()

In [5]:
img_proc = CLIPImageProcessor.from_pretrained(model_name)

In [7]:
import numpy as np
imgs = []
for i in range(1000):
    print(f"\r{i+1}/{1000}",end="")
    imgs.append(torch.tensor(np.array(img_proc(x[i*10:(i+1)*10])['pixel_values'])))

1000/1000

In [ ]:
#imgs_torch = [x for x in imgs]
x_train = torch.stack(imgs)

In [10]:
imgs[0].shape

(3, 224, 224)

In [20]:
imgs = [x['pixel_values'] for x in imgs]

In [ ]:
torch.tensor(imgs).shape

In [ ]:
x_train.shape